# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,76.17,74,100,12.39,PN,1706389875
1,1,luderitz,-26.6481,15.1594,65.70,89,100,5.59,NaN,1706389875
2,2,oktyabr'skoye,43.0645,44.7417,28.69,100,100,2.24,RU,1706389875
3,3,padangsidempuan,1.3667,99.2667,71.78,98,100,3.27,ID,1706389876
4,4,constantia,44.1833,28.6500,36.70,76,100,16.22,RO,1706389876


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City")

# Display the map plot
map_plot_1

# Display the map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>65) & (city_data_df["Max Temp"]<80)
                                  & (city_data_df["Wind Speed"]<10) & (city_data_df["Cloudiness"]<5)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()
ideal_cities_df.reset_index(inplace=True)
del ideal_cities_df["index"]
# Display sample data

ideal_cities_df.tail()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,330,san julian,21.0167,-102.1667,75.56,12,0,3.80,MX,1706389937
6,359,frontera,26.9333,-101.4500,67.32,41,2,5.01,MX,1706389943
7,386,puerto deseado,-47.7503,-65.8938,67.87,41,0,1.79,AR,1706389948
8,419,el granada,37.5027,-122.4694,66.63,72,0,5.75,US,1706389953
9,491,antsiranana,-12.2787,49.2917,75.07,87,0,6.62,MG,1706389967


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_cities_df[["City","Country","Lat","Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

/var/folders/vz/v1frsmss5gbchnkmsfv_w6fh0000gn/T/ipykernel_87055/2442310277.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,lompoc,US,34.6391,-120.4579,50,
1,maintirano,MG,-18.0667,44.0167,68,
2,pisco,PE,-13.7000,-76.2167,73,
3,ikom,NG,5.9667,8.7063,26,
4,morehead city,US,34.7229,-76.7260,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accomodation.hotel",
    "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lompoc - nearest hotel: No hotel found
maintirano - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
ikom - nearest hotel: No hotel found
morehead city - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
frontera - nearest hotel: No hotel found
puerto deseado - nearest hotel: No hotel found
el granada - nearest hotel: No hotel found
antsiranana - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,lompoc,US,34.6391,-120.4579,50,No hotel found
1,maintirano,MG,-18.0667,44.0167,68,No hotel found
2,pisco,PE,-13.7000,-76.2167,73,No hotel found
3,ikom,NG,5.9667,8.7063,26,No hotel found
4,morehead city,US,34.7229,-76.7260,85,No hotel found
5,san julian,MX,21.0167,-102.1667,12,No hotel found
6,frontera,MX,26.9333,-101.4500,41,No hotel found
7,puerto deseado,AR,-47.7503,-65.8938,41,No hotel found
8,el granada,US,37.5027,-122.4694,72,No hotel found
9,antsiranana,MG,-12.2787,49.2917,87,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name", "Country"])
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)